## 2025_Fall_DL_week5_Homework
8/7 CNN 실습 예제코드입니다!

이번 5주차 숙제는 총 2가지로 구성되어 있습니다.
- 논문 리뷰 - ResNet : https://arxiv.org/abs/1512.03385


    다음 논문을 읽고 4분이서 의견 공유 및 리뷰 발표를 준비해주시면 됩니다! (발표자, 자료, 시간 등 제약은 없으나, 1~2명이서 약 5분~10분정도로 notion이나 word를 이용하여 작성 후 pdf로 변환해서 깃허브에 제출해 주시는걸 추천합니다!)

- 코드 구현 - CNN

    아래 실습코드를 그대로 참고하셔도 좋고, 새롭게 짜셔도 좋습니다. 데이터셋은 CIFAR10이 아닌 벤치마크(MNIST, FFHQ, ImageNet)이나 다른 데이터셋을 사용하되, 학습 시간 및 리소스를 적절히 사용할 수 있는 화질을 사용하는 것을 추천 드립니다. 또한, 앞서 배운 regularization, initialization, optimizer 등등 기법을 추가해보시거나, layer를 변형하는 시도를 추가하여 결과를 분석해주세요. (스터디를 통해 각자한 시도가 겹치지 않으면 좋은 경험이 될 것 같습니다!)

    example : Earlystopping 추가, Dropoutlayer 추가, batch nomalization 추가, stride 및 padding 변형, Conv layer 추가 및 삭제 등등

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.adam import Adam
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
from tqdm import tqdm

In [7]:
from torchvision import datasets
# SVHN 데이터 사용 (Street View House Numbers, 32×32 컬러, 숫자 10클래스 0–9)

svhn_train = datasets.SVHN(root="./data", split="train", download=True)
svhn_test  = datasets.SVHN(root="./data", split="test",  download=True)

print(f"Train set: {len(svhn_train)} samples")
print(f"Test set: {len(svhn_test)} samples")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Train set: 73257 samples
Test set: 26032 samples
Using device: cuda


In [32]:
BATCH_SIZE = 128
EPOCHS = 20
LR = 1e-3

SVHN_MEAN = (0.4377, 0.4438, 0.4728)
SVHN_STD  = (0.1980, 0.2010, 0.1970)

# 데이터셋 로드 및 변환 적용
train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(SVHN_MEAN, SVHN_STD),
])

test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(SVHN_MEAN, SVHN_STD),
])

# Use datasets.SVHN directly and pass the transforms
train_set = datasets.SVHN(root="./data", split="train", download=True, transform=train_tf)
test_set  = datasets.SVHN(root="./data", split="test",  download=True, transform=test_tf)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_set,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)


print(f"Train set: {len(train_set)} samples")
print(f"Test set: {len(test_set)} samples")

Train set: 73257 samples
Test set: 26032 samples


## Modeling

In [34]:
class SmallCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        # CNN을 구성할 때 가장 먼저 입력 데이터 크기를 생각함 → SVHN은 (3, 32, 32) 컬러 이미지
        # Conv → BN → ReLU를 반복하고, 중간에 MaxPool로 공간 크기를 절반씩 줄이는 구조

        self.features = nn.Sequential(
            # 첫 번째 블록: 채널 3 → 32
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # 입력 크기 유지 (32x32)
            nn.BatchNorm2d(32),                          # 학습 안정화 및 수렴 속도 향상
            nn.ReLU(inplace=True),                       # 비선형성 추가

            # 같은 크기에서 채널 확장 없이 한 번 더 Conv
            nn.Conv2d(32, 32, kernel_size=3, padding=1), # 여전히 (32x32)
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # 공간 크기 절반 → (16x16)

            # 두 번째: 채널 32 → 64
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # (16x16)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 64, kernel_size=3, padding=1), # (16x16)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # 공간 절반 → (8x8)

            # 세 번째: 채널 64 → 128
            nn.Conv2d(64, 128, kernel_size=3, padding=1),# (8x8)
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                             # 공간 절반 → (4x4)
        )
        # 여기까지 거치면 feature map 크기가 (128채널, 4, 4) → 총 2048개의 feature임!

        self.classifier = nn.Sequential(
            nn.Flatten(),                                # 2D feature map을 1D 벡터로
            nn.Linear(128 * 4 * 4, 256),                 # 차원 축소
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),                           # 과적합 방지
            nn.Linear(256, num_classes)                  # 최종 출력층 → 클래스 개수만큼
        )

    def forward(self, x):
        # 특징 추출 단계
        x = self.features(x)
        # 분류 단계
        return self.classifier(x)

# 모델 인스턴스 생성 → num_classes=10 (SVHN은 0~9 숫자니까)
model = SmallCNN(num_classes=10).to(device)

# 다중 클래스 분류 >> CrossEntropy
criterion = nn.CrossEntropyLoss()

# 옵티마이저 Adam
optimizer = optim.Adam(model.parameters(), lr=LR)

# 학습률 스케줄러: 8에폭마다 학습률을 절반으로 → 후반부 세밀하게
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.5)

In [35]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()  # 학습 모드
    total_loss, total_correct, total_samples = 0.0, 0, 0

    for images, targets in loader:
        images, targets = images.to(device), targets.to(device)  # GPU/CPU로 이동

        optimizer.zero_grad()            # 기울기 초기화
        outputs = model(images)          # 순전파
        loss = criterion(outputs, targets)  # 손실 계산
        loss.backward()                  # 역전파
        optimizer.step()                  # 파라미터 업데이트

        batch_size = targets.size(0)
        total_loss += loss.item() * batch_size  # 배치별 손실 합산
        total_correct += outputs.argmax(1).eq(targets).sum().item()  # 맞춘 개수
        total_samples += batch_size

    epoch_loss = total_loss / total_samples  # 평균 손실
    epoch_acc  = total_correct / total_samples  # 정확도
    return epoch_loss, epoch_acc


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()  # 평가 모드
    total_loss, total_correct, total_samples = 0.0, 0, 0

    for images, targets in loader:
        images, targets = images.to(device), targets.to(device)  # GPU/CPU로 이동
        outputs = model(images)           # 순전파
        loss = criterion(outputs, targets)  # 손실 계산

        batch_size = targets.size(0)
        total_loss += loss.item() * batch_size  # 손실 합산
        total_correct += outputs.argmax(1).eq(targets).sum().item()  # 맞춘 개수
        total_samples += batch_size

    epoch_loss = total_loss / total_samples  # 평균 손실
    epoch_acc  = total_correct / total_samples  # 정확도
    return epoch_loss, epoch_acc

In [36]:
best_acc = 0.0
best_path = "svhn_smallcnn_best.pt"

#베스트 모델 저장 후 해당 모델로 학습하는 코드 feat. gpt


for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    test_loss,  test_acc  = evaluate(model, test_loader, criterion, device)
    scheduler.step()

    print(f"[{epoch:02d}/{EPOCHS}] "
          f"train loss {train_loss:.4f} | acc {train_acc*100:.2f}% || "
          f"test loss {test_loss:.4f} | acc {test_acc*100:.2f}%")

    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), best_path)

print(f"Best test acc: {best_acc*100:.2f}% (saved to {best_path})")

[01/20] train loss 1.5757 | acc 43.72% || test loss 0.8519 | acc 70.95%
[02/20] train loss 0.9261 | acc 67.97% || test loss 0.5944 | acc 80.57%
[03/20] train loss 0.7695 | acc 74.32% || test loss 0.5349 | acc 83.19%
[04/20] train loss 0.6914 | acc 76.97% || test loss 0.4295 | acc 86.72%
[05/20] train loss 0.6396 | acc 78.74% || test loss 0.4120 | acc 87.88%
[06/20] train loss 0.5987 | acc 80.20% || test loss 0.4151 | acc 87.77%
[07/20] train loss 0.5689 | acc 81.34% || test loss 0.3807 | acc 88.71%
[08/20] train loss 0.5477 | acc 82.05% || test loss 0.3428 | acc 89.66%
[09/20] train loss 0.4886 | acc 84.47% || test loss 0.2932 | acc 91.65%
[10/20] train loss 0.4704 | acc 85.03% || test loss 0.3019 | acc 91.37%
[11/20] train loss 0.4539 | acc 85.61% || test loss 0.2953 | acc 91.32%
[12/20] train loss 0.4445 | acc 85.86% || test loss 0.2982 | acc 91.45%
[13/20] train loss 0.4256 | acc 86.84% || test loss 0.2911 | acc 91.64%
[14/20] train loss 0.4201 | acc 87.24% || test loss 0.2866 | acc